In [1]:
import os

print("原始工作路径:", os.getcwd())
os.chdir("/workspace/")
print("新工作路径:", os.getcwd())

原始工作路径: /
新工作路径: /workspace


In [2]:
!apt-get -y update
!apt-get -y upgrade
!apt-get install -y cmake
!apt-get install -y libcurl4-gnutls-dev

Get:1 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2004/x86_64  InRelease [1581 B]
Get:2 http://mirrors.cloud.tencent.com/ubuntu focal InRelease [265 kB]         
Get:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2004/x86_64  Packages [2758 kB]
Get:4 http://mirrors.cloud.tencent.com/ubuntu focal-security InRelease [128 kB]
Get:5 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu focal InRelease [18.1 kB]
Get:6 http://mirrors.cloud.tencent.com/ubuntu focal-updates InRelease [128 kB]
Get:7 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu focal/main amd64 Packages [34.8 kB]
Get:8 http://mirrors.cloud.tencent.com/ubuntu focal-security/multiverse Sources [25.2 kB]
Get:9 http://mirrors.cloud.tencent.com/ubuntu focal-security/universe Sources [318 kB]
Get:10 http://mirrors.cloud.tencent.com/ubuntu focal-security/main Sources [423 kB]
Get:11 http://mirrors.cloud.tencent.com/ubuntu focal-security/restricted Sources [70.2 kB]
Get:12 http://mirrors.cloud.tencent.com

In [4]:
%pip install --upgrade pip
%pip install --upgrade ipywidgets

Looking in indexes: http://mirrors.tencentyun.com/pypi/simple
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 1.6 MB/s eta 0:00:00a 0:00:010m
  Attempting uninstall: pip
    Found existing installation: pip 25.0.1
    Uninstalling pip-25.0.1:
      Successfully uninstalled pip-25.0.1
Note: you may need to restart the kernel to use updated packages.
Looking in indexes: http://mirrors.tencentyun.com/pypi/simple
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 1.8 MB/s  0:00:01 eta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3/3 [ipywidgets]
Note: you may need to restart the kernel to use updated packages.


In [2]:
from unsloth import FastLanguageModel
import torch

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/Qwen2.5-14B-Instruct-bnb-4bit",
    load_in_4bit = True,
    use_gradient_checkpointing = True,
)

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
INFO 09-05 10:03:21 [__init__.py:241] Automatically detected platform cuda.
ERROR 09-05 10:03:22 [fa_utils.py:57] Cannot use FA version 2 is not supported due to FA2 is only supported on devices with compute capability >= 8
🦥 Unsloth Zoo will now patch everything to make training faster!
==((====))==  Unsloth 2025.8.10: Fast Qwen2 patching. Transformers: 4.55.4. vLLM: 0.10.1.1.
   \\   /|    Tesla V100-SXM2-32GB. Num GPUs = 1. Max memory: 31.739 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.7.1+cu126. CUDA: 7.0. CUDA Toolkit: 12.6. Triton: 3.3.1
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.31. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

In [3]:
model = FastLanguageModel.get_peft_model(
   model,
   r = 16, 
   target_modules = ["q_proj", "k_proj", "v_proj", "o_proj","gate_proj", "up_proj", "down_proj",],
   lora_alpha = 16,
   lora_dropout = 0, 
   bias = "none",    
   use_gradient_checkpointing = "unsloth", 
   random_state = 3407,
   use_rslora = False, 
   loftq_config = None, 
)

Unsloth 2025.8.10 patched 48 layers with 48 QKV layers, 48 O layers and 48 MLP layers.


In [4]:
from datasets import load_dataset
dataset = load_dataset('json', data_files='med-train-dateset.json', split='train')
chat_template = """下面是一些描述一些任务的说明。编写适当完成每个请求的响应。

### 指令:
{INPUT}

### 回应:
{OUTPUT}"""

from unsloth import apply_chat_template

dataset = apply_chat_template(
    dataset,
    tokenizer=tokenizer,
    chat_template=chat_template,
   
)
print(dataset[0])

Unsloth: We automatically added an EOS token to stop endless generations.


Map:   0%|          | 0/119397 [00:00<?, ? examples/s]

{'conversations': [{'content': '检查所见L3-S1CT平扫L3-4间盘膨出并见少量钙化影，硬膜囊受压。L4-5间盘膨出，硬膜囊受压。L5-S1间盘膨出，硬膜囊受压。双侧黄韧带不厚。骨性椎管不窄。定位像示腰椎骨质增生。', 'role': 'user'}, {'content': '考虑是骨质增生的情况你的情况考虑是老年骨骼退行性病变，可以补用钙剂，如疼痛严重考虑手术治疗。', 'role': 'assistant'}], 'text': '下面是一些描述一些任务的说明。编写适当完成每个请求的响应。\n\n### 指令:\n检查所见L3-S1CT平扫L3-4间盘膨出并见少量钙化影，硬膜囊受压。L4-5间盘膨出，硬膜囊受压。L5-S1间盘膨出，硬膜囊受压。双侧黄韧带不厚。骨性椎管不窄。定位像示腰椎骨质增生。\n\n### 回应:\n考虑是骨质增生的情况你的情况考虑是老年骨骼退行性病变，可以补用钙剂，如疼痛严重考虑手术治疗。<|im_end|>'}


In [5]:
from trl import SFTTrainer
from transformers import TrainingArguments
from unsloth import is_bfloat16_supported

trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = dataset,
    dataset_text_field = "text",
    max_seq_length = 2048,
    dataset_num_proc = 2,
    packing = False, 
    args = TrainingArguments(
        per_device_train_batch_size = 2,
        gradient_accumulation_steps = 4,
        warmup_steps = 5,
        max_steps = 60,
        learning_rate = 2e-4,
        fp16 = not is_bfloat16_supported(),
        bf16 = is_bfloat16_supported(),
        logging_steps = 1,
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = 3407,
        output_dir = "outputs",
        report_to = "none", 
    ),
)

Unsloth: Tokenizing ["text"] (num_proc=12):   0%|          | 0/119397 [00:00<?, ? examples/s]

Detected kernel version 5.4.0, which is below the recommended minimum of 5.5.0; this can cause the process to hang. It is recommended to upgrade the kernel to the minimum version or higher.


In [6]:
trainer_stats = trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 119,397 | Num Epochs = 1 | Total steps = 80
O^O/ \_/ \    Batch size per device = 2 | Gradient accumulation steps = 4
\        /    Data Parallel GPUs = 1 | Total batch size (2 x 4 x 1) = 8
 "-____-"     Trainable parameters = 68,812,800 of 14,838,846,464 (0.46% trained)


Unsloth: Will smartly offload gradients to save VRAM!


Step,Training Loss
1,3.541000
2,3.742100
3,3.471900
4,3.716400
5,2.998400
6,3.183900
7,2.861800
8,2.338000
9,2.438200
10,2.403100


In [8]:
FastLanguageModel.for_inference(model) 
messages = [                   
    {"role": "user", "content": "我肚子疼"},
]
input_ids = tokenizer.apply_chat_template(
    messages,
    add_generation_prompt = True,
    return_tensors = "pt",
).to("cuda")

from transformers import TextStreamer
text_streamer = TextStreamer(tokenizer, skip_prompt = True)
_ = model.generate(input_ids, streamer = text_streamer, max_new_tokens = 128, pad_token_id = tokenizer.eos_token_id)

这种情况是考虑肠炎或阑尾炎的可能性大，建议去医院检查明确诊断，如果是阑尾炎的话需要手术治疗，以免引起腹膜炎等严重并发症<|im_end|>


In [9]:
model.save_pretrained_merged("qwen25-14b-offical-finetuned-bnb-4bit", tokenizer, save_method="merged_4bit_forced",)

Unsloth: Merging LoRA weights into 4bit model...


/root/miniforge3/lib/python3.11/site-packages/peft/tuners/lora/bnb.py:348: UserWarning: Merge lora module to 4-bit linear may get different generations due to rounding errors.
  warnings.warn(


Unsloth: Merging finished.
Unsloth: Found skipped modules: ['model.layers.5.mlp.gate_proj', 'model.layers.5.mlp.up_proj', 'model.layers.5.mlp.down_proj', 'model.layers.23.self_attn.q_proj', 'model.layers.23.self_attn.k_proj', 'model.layers.23.self_attn.v_proj', 'model.layers.23.self_attn.o_proj', 'model.layers.24.mlp.gate_proj', 'model.layers.24.mlp.up_proj', 'model.layers.24.mlp.down_proj', 'model.layers.44.mlp.gate_proj', 'model.layers.44.mlp.up_proj', 'model.layers.44.mlp.down_proj', 'model.layers.46.mlp.gate_proj', 'model.layers.46.mlp.up_proj', 'model.layers.46.mlp.down_proj', 'lm_head']. Updating config.
Unsloth: Saving merged 4bit model to qwen25-14b-offical-finetuned-bnb-4bit...
Unsloth: Merged 4bit model saved.
Unsloth: Merged 4bit model process completed.
